# Initial Exploratory Analysis

## Setup

In [1]:
try:
    from google.colab import drive # type: ignore
    %pip install transformer_lens
    %pip install gdown
    # %pip install plotly
    # %pip install jaxtyping
    # %pip install einops
    # %pip install protobuf==3.20.*
    import os
    import sys
    from pathlib import Path
    import gdown
    if not Path("ioi_dataset.py").resolve().exists():
        urls = {
            "ioi_dataset.py": "https://drive.google.com/uc?id=19UjxFnb6kztuhvz6dGAXjA9oRZmd84kC",
            "path_patching.py": "https://drive.google.com/uc?id=1duF7B3IjG_E5nGcjT_BuoSrSkynUhZI5",
        }
        for filename, url in urls.items():
            output = str(Path(filename).resolve())
            gdown.download(url, output)
except:
    from IPython import get_ipython
    ipython = get_ipython()
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

In [2]:
import os
import pathlib
from typing import List, Optional, Union

import torch
import numpy as np
import yaml

import einops
from fancy_einsum import einsum

from datasets import load_dataset
#from transformers import pipeline

import transformers
import circuitsvis as cv
from transformers import AutoConfig, AutoModel, AutoModelForCausalLM
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

from torch import Tensor
from tqdm.notebook import tqdm
from jaxtyping import Float, Int, Bool
from typing import List, Optional, Callable, Tuple, Dict, Literal, Set
from rich import print as rprint

from typing import List, Union
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re

from functools import partial

from torchtyping import TensorType as TT

In [3]:
torch.set_grad_enabled(False)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [20]:
#update_layout_set = {"xaxis_range", "yaxis_range", "hovermode", "xaxis_title", "yaxis_title", "colorbar", "colorscale", "coloraxis", "title_x", "bargap", "bargroupgap", "xaxis_tickformat", "yaxis_tickformat", "title_y", "legend_title_text", "xaxis_showgrid", "xaxis_gridwidth", "xaxis_gridcolor", "yaxis_showgrid", "yaxis_gridwidth", "yaxis_gridcolor", "showlegend", "xaxis_tickmode", "yaxis_tickmode", "margin", "xaxis_visible", "yaxis_visible", "bargap", "bargroupgap"}

# def imshow(tensor, renderer=None, **kwargs):
#     kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
#     kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
#     facet_labels = kwargs_pre.pop("facet_labels", None)
#     border = kwargs_pre.pop("border", False)
#     if "color_continuous_scale" not in kwargs_pre:
#         kwargs_pre["color_continuous_scale"] = "RdBu"
#     if "margin" in kwargs_post and isinstance(kwargs_post["margin"], int):
#         kwargs_post["margin"] = dict.fromkeys(list("tblr"), kwargs_post["margin"])
#     fig = px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, **kwargs_pre).update_layout(**kwargs_post)
#     if facet_labels:
#         for i, label in enumerate(facet_labels):
#             fig.layout.annotations[i]['text'] = label
#     if border:
#         fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
#         fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
#     fig.show(renderer=renderer)

In [4]:
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def two_lines(tensor1, tensor2, renderer=None, **kwargs):
    px.line(y=[utils.to_numpy(tensor1), utils.to_numpy(tensor2)], **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

In [5]:
def get_logit_diff(logits, answer_token_indices, per_prompt=False):
    """Gets the difference between the logits of the provided tokens (e.g., the correct and incorrect tokens in IOI)

    Args:
        logits (torch.Tensor): Logits to use.
        answer_token_indices (torch.Tensor): Indices of the tokens to compare.

    Returns:
        torch.Tensor: Difference between the logits of the provided tokens.
    """
    if len(logits.shape) == 3:
        # Get final logits only
        logits = logits[:, -1, :]
    left_logits = logits.gather(1, answer_token_indices[:, 0].unsqueeze(1))
    right_logits = logits.gather(1, answer_token_indices[:, 1].unsqueeze(1))
    if per_prompt:
        print(left_logits - right_logits)

    return (left_logits - right_logits).mean()

## Exploratory Analysis


In [6]:
#source_model = AutoModelForCausalLM.from_pretrained("lvwerra/gpt2-imdb")
#rlhf_model = AutoModelForCausalLM.from_pretrained("curt-tigges/gpt2-negative-movie-reviews")

#hooked_source_model = HookedTransformer.from_pretrained(model_name="gpt2", hf_model=source_model)
#model = HookedTransformer.from_pretrained(model_name="EleutherAI/pythia-410m")
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


### Initial Examination

In [7]:
example_prompt = "I thought this movie was lousy, I hated it. \nConclusion: This movie is"
example_answer = " terrible"

In [8]:
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True, top_k=10)

Tokenized prompt: ['<|endoftext|>', 'I', ' thought', ' this', ' movie', ' was', ' lousy', ',', ' I', ' hated', ' it', '.', ' ', '\n', 'Conclusion', ':', ' This', ' movie', ' is']
Tokenized answer: [' terrible']


Performance on answer token:
Rank: 1        Logit: 14.10 Prob:  4.47% Token: | terrible|

Top 0th token. Logit: 14.87 Prob:  9.68% Token: | a|
Top 1th token. Logit: 14.10 Prob:  4.47% Token: | terrible|
Top 2th token. Logit: 13.93 Prob:  3.77% Token: | bad|
Top 3th token. Logit: 13.90 Prob:  3.67% Token: | so|
Top 4th token. Logit: 13.87 Prob:  3.57% Token: | not|
Top 5th token. Logit: 13.80 Prob:  3.30% Token: | awful|
Top 6th token. Logit: 13.28 Prob:  1.98% Token: | pretty|
Top 7th token. Logit: 13.25 Prob:  1.91% Token: | just|
Top 8th token. Logit: 13.16 Prob:  1.75% Token: | really|
Top 9th token. Logit: 13.13 Prob:  1.69% Token: | the|


Ranks of the answer tokens: [(' terrible', 1)]

In [9]:
example_prompt = "I thought this movie was amazing, I loved it. \nConclusion: This movie is"
example_answer = " amazing"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True, top_k=10)

Tokenized prompt: ['<|endoftext|>', 'I', ' thought', ' this', ' movie', ' was', ' amazing', ',', ' I', ' loved', ' it', '.', ' ', '\n', 'Conclusion', ':', ' This', ' movie', ' is']
Tokenized answer: [' amazing']


Performance on answer token:
Rank: 1        Logit: 14.65 Prob:  5.62% Token: | amazing|

Top 0th token. Logit: 15.37 Prob: 11.47% Token: | a|
Top 1th token. Logit: 14.65 Prob:  5.62% Token: | amazing|
Top 2th token. Logit: 14.58 Prob:  5.23% Token: | so|
Top 3th token. Logit: 14.27 Prob:  3.82% Token: | awesome|
Top 4th token. Logit: 14.25 Prob:  3.75% Token: | great|
Top 5th token. Logit: 14.21 Prob:  3.58% Token: | one|
Top 6th token. Logit: 14.06 Prob:  3.09% Token: | really|
Top 7th token. Logit: 13.91 Prob:  2.67% Token: | the|
Top 8th token. Logit: 13.82 Prob:  2.43% Token: | not|
Top 9th token. Logit: 13.75 Prob:  2.27% Token: | just|


Ranks of the answer tokens: [(' amazing', 1)]

### Dataset Construction

In [10]:
positive_adjectives = [
    ' perfect', ' fantastic',' delightful',' cheerful',' marvelous',' good',' remarkable',' wonderful',
    ' fabulous',' outstanding',' awesome',' exceptional',' incredible',' extraordinary',
    ' amazing',' lovely',' brilliant',' charming',' terrific',' superb',' spectacular',' great',' splendid',
    ' beautiful',' joyful',' positive',' excellent'
    ]

negative_adjectives = [
    ' dreadful',' bad',' dull',' depressing',' miserable',' tragic',' nasty',' inferior',' horrific',' terrible',
    ' ugly',' disgusting',' disastrous',' horrendous',' annoying',' boring',' offensive',' frustrating',' wretched',' dire',
    ' awful',' unpleasant',' horrible',' mediocre',' disappointing',' inadequate'
    ]

#negative_adjectives = [' lousy', ' dire', ' bad', ' nasty', ' miserable', ' wretched', ' disgusting', ' ugly', ' disastrous', ' tragic']

len(positive_adjectives), len(negative_adjectives)


(27, 26)

In [11]:
all_prompts = []

pos_prompts = [
    f"I thought this movie was{positive_adjectives[i]}, I loved it. \nConclusion: This movie is" for i in range(len(positive_adjectives)-1)
]
neg_prompts = [
    f"I thought this movie was{negative_adjectives[i]}, I hated it. \nConclusion: This movie is" for i in range(len(negative_adjectives)-1)
]
# List of the token (ie an integer) corresponding to each answer, in the format (correct_token, incorrect_token)
answer_tokens = []
for i in range(len(pos_prompts)-1):

    all_prompts.append(pos_prompts[i])
    all_prompts.append(neg_prompts[i])
    
    answer_tokens.append(
        (
            model.to_single_token(" amazing"),
            model.to_single_token(" terrible"),
        )
    )

    answer_tokens.append(
        (
            model.to_single_token(" terrible"),
            model.to_single_token(" amazing"),
        )
    )

answer_tokens = torch.tensor(answer_tokens).to(device)

prompts_tokens = model.to_tokens(all_prompts, prepend_bos=True)
clean_tokens = prompts_tokens.to(device)

corrupted_tokens = model.to_tokens(all_prompts[1:] + list(all_prompts[0]), prepend_bos=True)

In [46]:
for i in range(len(all_prompts)):
    logits, _ = model.run_with_cache(all_prompts[i])
    print(all_prompts[i])
    print(get_logit_diff(logits, answer_tokens[i].unsqueeze(0)))

I thought this movie was perfect, I loved it. 
Conclusion: This movie is
tensor(2.3302, device='cuda:0')
I thought this movie was dreadful, I hated it. 
Conclusion: This movie is
tensor(2.4427, device='cuda:0')
I thought this movie was fantastic, I loved it. 
Conclusion: This movie is
tensor(2.3365, device='cuda:0')
I thought this movie was bad, I hated it. 
Conclusion: This movie is
tensor(2.0416, device='cuda:0')
I thought this movie was delightful, I loved it. 
Conclusion: This movie is
tensor(1.8997, device='cuda:0')
I thought this movie was dull, I hated it. 
Conclusion: This movie is
tensor(0.7407, device='cuda:0')
I thought this movie was cheerful, I loved it. 
Conclusion: This movie is
tensor(1.3165, device='cuda:0')
I thought this movie was depressing, I hated it. 
Conclusion: This movie is
tensor(1.5101, device='cuda:0')
I thought this movie was marvelous, I loved it. 
Conclusion: This movie is
tensor(2.5645, device='cuda:0')
I thought this movie was miserable, I hated it. 
C

In [13]:
clean_logits, clean_cache = model.run_with_cache(clean_tokens)
clean_logit_diff = get_logit_diff(clean_logits, answer_tokens, per_prompt=False)
clean_logit_diff

tensor(2.0828, device='cuda:0')

In [14]:
corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens)
corrupted_logit_diff = get_logit_diff(corrupted_logits, answer_tokens, per_prompt=False)
corrupted_logit_diff

tensor(-2.0904, device='cuda:0')

### Direct Logit Attribution

In [15]:
answer_residual_directions = model.tokens_to_residual_directions(answer_tokens)
print("Answer residual directions shape:", answer_residual_directions.shape)
logit_diff_directions = answer_residual_directions[:, 0] - answer_residual_directions[:, 1]
print("Logit difference directions shape:", logit_diff_directions.shape)

Answer residual directions shape: torch.Size([50, 2, 768])
Logit difference directions shape: torch.Size([50, 768])


In [18]:
# cache syntax - resid_post is the residual stream at the end of the layer, -1 gets the final layer. The general syntax is [activation_name, layer_index, sub_layer_type]. 
final_residual_stream = clean_cache["resid_post", -1]
print("Final residual stream shape:", final_residual_stream.shape)
final_token_residual_stream = final_residual_stream[:, -1, :]
# Apply LayerNorm scaling
# pos_slice is the subset of the positions we take - here the final token of each prompt
scaled_final_token_residual_stream = clean_cache.apply_ln_to_stack(final_token_residual_stream, layer = -1, pos_slice=-1)

average_logit_diff = einsum("batch d_model, batch d_model -> ", scaled_final_token_residual_stream, logit_diff_directions)/len(all_prompts)
print("Calculated average logit diff:", average_logit_diff.item())
print("Original logit difference:",clean_logit_diff.item())

Final residual stream shape: torch.Size([50, 19, 768])
Calculated average logit diff: 2.082763671875
Original logit difference: 2.082763671875


#### Logit Lens

In [19]:
def residual_stack_to_logit_diff(residual_stack: TT["components", "batch", "d_model"], cache: ActivationCache) -> float:
    scaled_residual_stack = clean_cache.apply_ln_to_stack(residual_stack, layer = -1, pos_slice=-1)
    return einsum("... batch d_model, batch d_model -> ...", scaled_residual_stack, logit_diff_directions)/len(all_prompts)

In [20]:
accumulated_residual, labels = clean_cache.accumulated_resid(layer=-1, incl_mid=True, pos_slice=-1, return_labels=True)
logit_lens_logit_diffs = residual_stack_to_logit_diff(accumulated_residual, clean_cache)
line(logit_lens_logit_diffs, x=np.arange(model.cfg.n_layers*2+1)/2, hover_name=labels, title="Logit Difference From Accumulate Residual Stream")

#### Layer Attribution

In [21]:
per_layer_residual, labels = clean_cache.decompose_resid(layer=-1, pos_slice=-1, return_labels=True)
per_layer_logit_diffs = residual_stack_to_logit_diff(per_layer_residual, clean_cache)

line(per_layer_logit_diffs, hover_name=labels, title="Logit Difference From Each Layer")

#### Head Attribution

In [22]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

per_head_residual, labels = clean_cache.stack_head_results(layer=-1, pos_slice=-1, return_labels=True)
per_head_logit_diffs = residual_stack_to_logit_diff(per_head_residual, clean_cache)
per_head_logit_diffs = einops.rearrange(per_head_logit_diffs, "(layer head_index) -> layer head_index", layer=model.cfg.n_layers, head_index=model.cfg.n_heads)
imshow(per_head_logit_diffs, labels={"x":"Head", "y":"Layer"}, title="Logit Difference From Each Head")

Tried to stack head results when they weren't cached. Computing head results now


### Circuit Sketching